In [ ]:
import sqlite3
import pandas as pd

In [ ]:
query = '''SELECT UserPPDataSetExperiment.ID AS 'DataSetID', UserPPAnalysisSet.Subset,
UserPPDataSetExperiment.PDBFileID AS UserPPDataSetExperimentPDBFileID,
PDBFile.Resolution,
PPComplex.LName,
PPComplex.RName,

PPMutagenesisMutation.RecordKey AS 'Mutations',
IFNULL(PositiveDDG.DDG, 0) - IFNULL(NegativeDDG.DDG, 0) AS ExperimentalDDG,
PPIPDBPartnerChain.*

FROM UserPPDataSetExperiment
INNER JOIN UserPPAnalysisSet ON UserPPAnalysisSet.UserPPDataSetExperimentID=UserPPDataSetExperiment.ID
INNER JOIN PPComplex ON PPComplex.ID=UserPPDataSetExperiment.PPComplexID
INNER JOIN PPMutagenesis ON PPMutagenesis.ID=UserPPAnalysisSet.PPMutagenesisID
INNER JOIN PPMutagenesisMutation ON PPMutagenesisMutation.PPMutagenesisID=PPMutagenesis.ID
INNER JOIN PDBFile ON PDBFile.ID=UserPPDataSetExperiment.PDBFileID
INNER JOIN PPIPDBPartnerChain ON PPIPDBPartnerChain.PPComplexID=PPComplex.ID AND PPIPDBPartnerChain.PDBFileID=UserPPDataSetExperiment.PDBFileID
LEFT JOIN PPIDDG AS PositiveDDG ON UserPPAnalysisSet.PositiveDependentPPIDDGID=PositiveDDG.ID
LEFT JOIN PPIDDG AS NegativeDDG ON UserPPAnalysisSet.NegativeDependentPPIDDGID=NegativeDDG.ID

WHERE UserPPAnalysisSet.Subset='ZEMu';'''

ddG_db = sqlite3.connect('170611-ddg_db_backup.db3')
df = pd.read_sql_query(query, ddG_db)
print(df['DataSetID'].nunique())
print(len(df))
df.head()

In [ ]:
data_df = []
for (data_set_id, pdb_id), group_df in df.groupby(['DataSetID', 'UserPPDataSetExperimentPDBFileID']):
    # print(data_set_id, pdb_id)
    chain_pdb_ids = group_df['PDBFileID'].drop_duplicates().values
    try:
        assert(len(chain_pdb_ids) == 1)
        assert(chain_pdb_ids[0] == pdb_id)
    except AssertionError:
        display(group_df)
        raise
    mutations = ';'.join(sorted(group_df['Mutations'].drop_duplicates()))
    l_chains = ';'.join(sorted(group_df.loc[group_df['Side'] == 'L', 'Chain'].drop_duplicates()))
    r_chains = ';'.join(sorted(group_df.loc[group_df['Side'] == 'R', 'Chain'].drop_duplicates()))
    data_df.append((data_set_id, pdb_id, mutations, l_chains, r_chains))
data_df = pd.DataFrame(data_df, columns=['DataSetID', 'PDBFileID', 'Mutations', 'LChains', 'RChains'])
print(len(data_df))
data_df.to_csv('ZEMu_chains_to_move.csv', index=False)
data_df.head(n=15)